In [1]:
from wikiparser_utils import WikiXMLDump, WikiPage
import os
import nltk
import json
import numpy as np
from tqdm import tqdm
from difflib import Differ 
from utils.difflibparser import DifflibParser, DiffCode
import shutil

import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from io import StringIO, BytesIO
from PyPDF2 import PdfReader, PdfFileReader
import urllib3
import html2text
import os
import time
from os import listdir
from os.path import isfile, join

from selenium.webdriver.chrome.options import Options
from urllib3.util import Timeout
timeout = Timeout(total=60.0)
http_urllib = urllib3.PoolManager(timeout=timeout)
options = Options()
options.headless = True

/home/ubuntu/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (4.0.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
dump = WikiXMLDump('data/history.xml')

DOCS_DIR = 'data/documents'
DOCS_MAPPER_DIR = 'data/documents_mapper'

if not os.path.exists(DOCS_MAPPER_DIR):
    os.makedirs(DOCS_MAPPER_DIR)
    
if not os.path.exists(DOCS_DIR):
    os.makedirs(DOCS_DIR)

dirs = sorted(os.listdir('data/pages'))

In [3]:
pbar = tqdm(desc='Downloading documents (total progress)', leave=True, position=0)
# dirs = dirs[:200]
for page_id_id, page_name in enumerate(dirs):
    page_docs_mapper_link2id = {}
    page_docs_mapper_id2link = {}
    
    page_revisions = WikiPage().load_revisions(f"data/pages/{page_name}")
    rev_list = sorted(list(map(int, list(page_revisions))))
    
    if os.path.exists(f'{DOCS_DIR}/{page_name}') and os.path.exists(f'{DOCS_MAPPER_DIR}/{page_name}'):
        try:
            page_docs_mapper_id2link_done = json.load(open(f'{DOCS_MAPPER_DIR}/{page_name}/id2link.json', "r"))
            page_docs_mapper_link2id_done = json.load(open(f'{DOCS_MAPPER_DIR}/{page_name}/link2id.json', "r"))
            ids_of_links = list(map(int, page_docs_mapper_link2id_done.values()))
            docs_names = list(map(lambda x: int(x.split('.')[0]), os.listdir(f'{DOCS_DIR}/{page_name}')))
            ids_ = list(map(int, page_docs_mapper_id2link_done))
            if set(docs_names) == set(ids_) and set(ids_of_links) == set(ids_):
                print(f'Page {page_name} is OK!')
                continue # go to next page
            else:
                print(f'Page {page_name} is NOT OK!')
        except:
            print(f'Page {page_name} is READ ERROR!')
            pass
    
    if not os.path.exists(f'{DOCS_DIR}/{page_name}'):
        os.makedirs(f'{DOCS_DIR}/{page_name}')
    else:
        shutil.rmtree(f'{DOCS_DIR}/{page_name}')
        os.makedirs(f'{DOCS_DIR}/{page_name}')
    current_link_id = 0
    for last_rev_num in rev_list[::-1]:
        last_rev = page_revisions[last_rev_num]
        docs_links, _ = last_rev.get_links()
        
        for link in docs_links:
            url_link = link[1]
            if url_link in page_docs_mapper_link2id:
                continue
                
            if 'pdf' == url_link[-3:]:
                try:
                    response = http_urllib.request('GET', url_link)
                    if str(response.status).strip() == '200':
                        memoryFile = BytesIO(response.data)
                        pdfFile = PdfFileReader(memoryFile)
                        answer = ''
                        for pageNum in range(pdfFile.getNumPages()):
                            currentPage = pdfFile.getPage(pageNum)
                            answer += currentPage.extract_text() + '\n'
                        if 'page not found' not in answer.lower() and len(answer) > 0:
                            with open(f'{DOCS_DIR}/{page_name}/{current_link_id}.txt', 'w', encoding='utf-8') as f:
                                f.write(answer)
                                page_docs_mapper_id2link[current_link_id] = url_link
                                page_docs_mapper_link2id[url_link] = current_link_id
                                current_link_id += 1
                    else:
                        pass
                except:
                    driver = webdriver.Chrome(options=options)
                    driver.set_page_load_timeout(60)
                    driver.get('https://pdfmyurl.com/')
                    a = driver.find_element(By.ID, 'url')
                    a.send_keys(url_link)
                    button_element = driver.find_element(By.NAME, "submit")
                    button_element.click()
                    time.sleep(10)
                    driver.close()


                    pdf_saved_path = ''
                    for f_n in [f for f in listdir('./') if isfile(join('./', f))]:
                        if 'pdf' in f_n:
                            pdf_saved_path = f"./{f_n}"
                            break
                    if pdf_saved_path:
                        reader = PdfReader(pdf_saved_path)
                        text = ""
                        for page in reader.pages:
                            text += page.extract_text() + "\n"
                        if 'page not found' not in text.lower() and len(text) > 0:
                            with open(f'{DOCS_DIR}/{page_name}/{current_link_id}.txt', 'w', encoding='utf-8') as f:
                                f.write(text)
                                page_docs_mapper_id2link[current_link_id] = url_link
                                page_docs_mapper_link2id[url_link] = current_link_id
                                current_link_id += 1
                        os.remove(pdf_saved_path)

            else:
                try:
                    response = requests.get(url_link, timeout=60)
                    if '200' in str(response):
                        text_html = response.text
                        clean_text = html2text.html2text(text_html)
                        if 'page not found' not in clean_text.lower() and len(clean_text) > 0:
                            with open(f'{DOCS_DIR}/{page_name}/{current_link_id}.txt', 'w', encoding='utf-8') as f:
                                f.write(clean_text)
                                page_docs_mapper_id2link[current_link_id] = url_link
                                page_docs_mapper_link2id[url_link] = current_link_id
                                current_link_id += 1
                    else:
                        driver = webdriver.Chrome(options=options)
                        driver.set_page_load_timeout(60)
                        driver.get(url_link)
                        text_html = driver.page_source
                        clean_text = html2text.html2text(text_html)
                        if 'page not found' not in clean_text.lower() and len(clean_text) > 0:
                            with open(f'{DOCS_DIR}/{page_name}/{current_link_id}.txt', 'w', encoding='utf-8') as f:
                                f.write(clean_text)
                                page_docs_mapper_id2link[current_link_id] = url_link
                                page_docs_mapper_link2id[url_link] = current_link_id
                                current_link_id += 1
                        driver.close()
                except:
                    try:
                        driver = webdriver.Chrome(options=options)
                        driver.set_page_load_timeout(60)
                        driver.get(url_link)
                        text_html = driver.page_source
                        clean_text = html2text.html2text(text_html)
                        if 'page not found' not in clean_text.lower() and len(clean_text) > 0:
                            with open(f'{DOCS_DIR}/{page_name}/{current_link_id}.txt', 'w', encoding='utf-8') as f:
                                f.write(clean_text)
                                page_docs_mapper_id2link[current_link_id] = url_link
                                page_docs_mapper_link2id[url_link] = current_link_id
                                current_link_id += 1
                        driver.close()
                    except:
                        pass
            pbar.update(1)
        pbar.update(1)
    
    print(f'Page {page_id_id} of {len(dirs)} processed      ----      {page_name}')
    if not os.path.exists(f'{DOCS_MAPPER_DIR}/{page_name}'):
        os.makedirs(f'{DOCS_MAPPER_DIR}/{page_name}')
    json.dump(page_docs_mapper_id2link, open(f'{DOCS_MAPPER_DIR}/{page_name}/id2link.json', "w"))
    json.dump(page_docs_mapper_link2id, open(f'{DOCS_MAPPER_DIR}/{page_name}/link2id.json', "w"))

Page 01JJC_2934682 is OK!
Page 10 OClock News_2934804 is OK!
Page 125 BC_2934966 is OK!
Page 15020421743_2935130 is OK!
Page 151199192110_2935750 is OK!
Page 15663101131_2935126 is OK!
Page 168169176113_2936145 is OK!
Page 16816988214_2935499 is OK!
Page 1701102431_2935480 is OK!
Page 1787 in rail transport_2934818 is OK!
Page 1922 Newport byelection_2935525 is OK!
Page 194171522_2934587 is OK!
Page 19425424117_2935210 is OK!
Page 19929215212_2936094 is OK!
Page 1995 American League Division Series_2935400 is OK!
Page 1998 Macedonian parliamentary election_2935076 is OK!
Page 1nt3rn3t_2934926 is OK!
Page 20 to 1 Most Embarrassing Moments_2934947 is OK!
Page 2000 ISSF World Cup_2934684 is OK!
Page 2001 ISSF World Cup_2934645 is OK!
Page 2003 Liberation of Iraq_2935500 is OK!
Page 2005 Conservative Party leadership election_2936109 is OK!
Page 202152170254_2934906 is OK!
Page 203113237127_2935487 is OK!
Page 2043972121_2935969 is OK!
Page 2048115392_2935502 is OK!
Page 2048122543_2935448

Page Cervical weakness_2935035 is OK!
Page Cervical weakness_2935038 is OK!
Page Channel 31 Australia_2935206 is OK!
Page Charles DouglasHome journalist_2934855 is OK!
Page Chicks on speed_2935739 is OK!
Page Chief cell_2936115 is OK!
Page Childcare Resources_2936242 is OK!
Page Chiral column chromatography_2936014 is OK!
Page Christian peoples Alliance_2936137 is OK!
Page Chuluunkhoroot_2935018 is OK!
Page Cinema of Austria_2935598 is OK!
Page Citizen James_2934707 is OK!
Page Clint Hurdle_2935720 is OK!
Page Clive Sullivan_2934971 is OK!
Page Closed and exact forms_2935139 is OK!
Page Closedmouth_2935590 is OK!
Page Closedmouth_2935596 is OK!
Page Coding efficiency_2935140 is OK!
Page Cognitive approaches to grammar_2935955 is OK!
Page Cognitive robotics_2934910 is OK!
Page Coldwater River Ontario_2934623 is OK!
Page Collaboration_2934593 is OK!
Page Colletes inaequalis_2934770 is OK!
Page Communist youth movement_2934890 is OK!
Page Companies Investigation Branch_2934831 is OK!
Page

KeyboardInterrupt: 

In [7]:
print(page_name)
json.dump(page_docs_mapper_id2link, open(f'{DOCS_MAPPER_DIR}/{page_name}/id2link.json', "w"))
json.dump(page_docs_mapper_link2id, open(f'{DOCS_MAPPER_DIR}/{page_name}/link2id.json', "w"))

Darkwood Dub_2935508
